In [ ]:
! pip install tiktoken

In [ ]:
! pip install transformers_stream_generator

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

model_name = 'Qwen-1_8B-Chat'
model_path = '/share/huggingface/'
device = 'auto'
tokenizer = AutoTokenizer.from_pretrained(model_path + model_name, trust_remote_code=True, device_map=device)
model = AutoModelForCausalLM.from_pretrained(model_path + model_name, trust_remote_code=True, device_map=device)
model

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-23): 24 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (core_attention_flash): FlashSelfAttention()
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear(in_features=2048, out_features=5504, bias=False)
          (w2): Linear(in_features=2048, out_features=5504, bias=False)
          (c_proj): Linear(in_features=5504, out_features=2048, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=151936, bias=False)
)

In [3]:
for k, v in model.named_parameters():
    if 'h.0' in k:
        print(k, v.shape, v.dtype)

transformer.h.0.ln_1.weight torch.Size([2048]) torch.bfloat16
transformer.h.0.attn.c_attn.weight torch.Size([6144, 2048]) torch.bfloat16
transformer.h.0.attn.c_attn.bias torch.Size([6144]) torch.bfloat16
transformer.h.0.attn.c_proj.weight torch.Size([2048, 2048]) torch.bfloat16
transformer.h.0.ln_2.weight torch.Size([2048]) torch.bfloat16
transformer.h.0.mlp.w1.weight torch.Size([5504, 2048]) torch.bfloat16
transformer.h.0.mlp.w2.weight torch.Size([5504, 2048]) torch.bfloat16
transformer.h.0.mlp.c_proj.weight torch.Size([2048, 5504]) torch.bfloat16


: 

In [ ]:
# 读取yaml文件
import yaml

with open('./gpt-j-6b.yaml', 'r') as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

hparams

In [ ]:
for bs in [8,16,32,64]:
    with open(f'./test/gpt-j-6b-bs-{bs}.yaml', 'w') as f:
        hparams['batch_size'] = bs
        yaml.dump(hparams, f)

In [ ]:
# target_modules
target_modules_dict ={
    'attn': ['q_proj','k_proj','v_proj','out_proj'],
    'mlp': ['fc_in','fc_out'],
    'all': ['q_proj','k_proj','v_proj','out_proj','fc_in','fc_out'],
}
for target_modules in target_modules_dict:
    print(target_modules_dict[target_modules])
    # layers
    for layer in range(0,28,7):
        print([i for i in range(layer,layer+7)])
        layers = [i for i in range(layer,layer+7)]
        with open(f'./test/gpt-j-6b-{target_modules}-{layers[0]}-{layers[-1]}.yaml', 'w') as f:
            hparams['layers'] = layers
            hparams['target_modules'] = target_modules_dict[target_modules]
            # hparams['bf16'] = True
            yaml.dump(hparams, f)

In [ ]:
with open('./test/gpt-j-6b-all-0-6.yaml', 'r') as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)
hparams